In [1]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from statistics import median, mean
from matplotlib import pyplot as plt
import numpy as np
import json
from tqdm import tqdm
from glob import glob
import os
import sys
sys.path.insert(0,"/study/mrphys/skunkworks/kk/mriUnet")
import unet
from torchvision import transforms
from torch.utils.data import Dataset
from sklearn.model_selection import KFold as kf
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
import h5py
import shutil

In [ ]:
def getComplexSlices(path):

    with h5py.File(path,'r') as hf:
        prefix = 'C_000_0'
        imagestackReal = []
        imagestackImag = []
        for i in range(10):
            n = prefix + str(i).zfill(2)
            image = hf['Images'][n]
            imagestackReal.append(np.array(image['real']))
            imagestackImag.append(np.array(image['imag']))
            if i==0:
                normScale = np.abs(np.array(image['real']+image['real']*1j)).max()
        imagestackReal = np.array(imagestackReal)/normScale
        imagestackImag = np.array(imagestackImag)/normScale
        
    return imagestackReal+imagestackImag*1j, normScale

In [4]:
allImages = sorted(glob("/study/mrphys/skunkworks/training_data//mover01/*/", recursive=True))[0:40]

In [5]:
def getComplexSlices(path):

    with h5py.File(path,'r') as hf:
        prefix = 'C_000_0'
        imagestackReal = []
        imagestackImag = []
        for i in range(10):
            n = prefix + str(i).zfill(2)
            image = hf['Images'][n]
            imagestackReal.append(np.array(image['real']))
            imagestackImag.append(np.array(image['imag']))
            if i==0:
                normScale = np.abs(np.array(image['real']+image['real']*1j)).max()
        imagestackReal = np.array(imagestackReal)/normScale
        imagestackImag = np.array(imagestackImag)/normScale
        
    return imagestackReal+imagestackImag*1j, normScale

class mriNoisyDataset(Dataset):
    def __init__(self, sample):
        self.originalPath = []
        self.accelPath = [] 

        allImages = sorted(glob("/study/mrphys/skunkworks/training_data//mover01/*/", recursive=True))[0:40]
        folderName  = allImages[sample]
        self.accelPath = folderName +'processed_data/acc_2min/C.h5'
        self.accelFile, self.scale = getComplexSlices(self.accelPath)

    def __getitem__(self, index):
        if index<256:
            return self.accelFile[:,index,:,:]
        elif index<512:
            index = index-256
            return self.accelFile[:,:,index,:]
        else:
            index = index-512
            return self.accelFile[:,:,:,index]
        
    def __len__(self):
        return 768

In [12]:
def predict(model, dataset, device = 5):
    model.eval()
    model.to(device)
    X = []
    Y = []
    Z = []
    for i, noisy in enumerate(dataset):
        noisy = torch.tensor(noisy).to(device).unsqueeze(0)
        with torch.no_grad():
            p = model(noisy).cpu().numpy() * dataset.scale
            if i<256:
                X.append(p)
            elif i<512:
                Y.append(p)
            else:
                Z.append(p)
                
    return np.vstack(X).transpose(1,0,2,3), np.vstack(Y).transpose(1,2,0,3), np.vstack(Z).transpose(1,2,3,0)

In [13]:
folds = 5
kfsplitter = kf(n_splits=folds, shuffle=True, random_state=69420)
for i, (train_index, test_index) in enumerate(kfsplitter.split(allImages)):
    fold = i+1
    model = unet.UNet(
        10,
        10,
        f_maps=32,
        layer_order=['separable convolution', 'relu'],
        depth=4,
        layer_growth=2.0,
        residual=True,
        complex_input=True,
        complex_kernel=True,
        ndims=2,
        padding=1
    )
    name = f'fullDenoiser_{fold}'
    model.load_state_dict(torch.load(f'/study/mrphys/skunkworks/kk/outputs/{name}/weights/{name}_BEST.pth'))
    for index in tqdm(test_index):
        dataset = mriNoisyDataset(index)
        X, Y, Z = predict(model, dataset)
        pred = (X+Y+Z)/3
        np.save(f'pred/denoised_{index}.npy',np.array(pred))

Crop amount [(-4, -4, -4, -4), (-16, -16, -16, -16), (-40, -40, -40, -40)]
Crop amount [(-4, -4, -4, -4), (-16, -16, -16, -16), (-40, -40, -40, -40)]
Crop amount [(-4, -4, -4, -4), (-16, -16, -16, -16), (-40, -40, -40, -40)]
Crop amount [(-4, -4, -4, -4), (-16, -16, -16, -16), (-40, -40, -40, -40)]
Crop amount [(-4, -4, -4, -4), (-16, -16, -16, -16), (-40, -40, -40, -40)]


  0%|                                                                                             | 0/2 [00:21<?, ?it/s]


KeyboardInterrupt: 

In [4]:
def getComplexSlices(path):

    with h5py.File(path,'r') as hf:
        prefix = 'C_000_0'
        imagestackReal = []
        imagestackImag = []
        for i in range(10):
            n = prefix + str(i).zfill(2)
            image = hf['Images'][n]
            imagestackReal.append(np.array(image['real']))
            imagestackImag.append(np.array(image['imag']))
            if i==0:
                normScale = np.abs(np.array(image['real']+image['imag']*1j)).max()
        imagestackReal = np.array(imagestackReal)/normScale
        imagestackImag = np.array(imagestackImag)/normScale
        
    return imagestackReal+imagestackImag*1j, normScale

T1path = sorted(glob('/study/mrphys/skunkworks/training_data/mover01/*/processed_data/T1_3_tv.nii'))
samples = [p.split('/')[6] for p in T1path]
noisyPath = [f'/study/mrphys/skunkworks/training_data/mover01/{p}/processed_data/acc_2min/C.h5' for p in samples]
for i, x in tqdm(enumerate(noisyPath)):
    d = f'/study/mrphys/skunkworks/kk/pred/denoised_{i}.npy'
    array = np.load(d)
    _, normScale = getComplexSlices(x)
    array *= normScale
    np.save(d, array)
    
allImages = sorted(glob('/study/mrphys/skunkworks/kk/pred/denoised_*.npy', recursive=True))
try:
    os.mkdir('/scratch/mrphys/denoised')
except:
    pass

for i, imgIndex in tqdm(enumerate(range(len(allImages)))):
    name = sorted(glob("/study/mrphys/skunkworks/training_data//mover01/*/", recursive=True))[imgIndex].split('/')[-2]
    
    try:
        os.mkdir(f'/scratch/mrphys/denoised/{name}')
    except:
        pass
    
    for file in ['basis.arma','gmpnrage_parameters.txt','mpnrage_t1fitting_parameters.txt']:
                
        shutil.copy(f'/study/mrphys/skunkworks/kk/modelfitting/{file}', f'/scratch/mrphys/denoised/{name}/{file}')

    with h5py.File(f'/scratch/mrphys/denoised/{name}/C.h5','w') as f:
        pred = np.load(f'pred/denoised_{i}.npy')
        temp = pred.astype(np.dtype([('real','f'),('imag','f')]))
        temp['imag'] = pred.imag
        pred = temp
        grp = f.create_group('Images')
        for n in range(16):     
            if n in list(range(10)):
                grp.create_dataset('C_000_0'+ str(n).zfill(2), data=pred[n])
            else:
                grp.create_dataset('C_000_0'+ str(n).zfill(2), data=pred[0]*0)
                
    break

3it [00:29,  9.84s/it]


KeyboardInterrupt: 

In [2]:
os.listdir(')

['C-Copy1.h5',
 '.ipynb_checkpoints',
 'basis.arma',
 'gmpnrage_parameters.txt',
 'mpnrage_t1fitting_parameters.txt']

In [2]:
import torch
x = torch.randn(3,10,256,256)

In [12]:
0.6746*1.4824

1.00002704

In [11]:
x

tensor([[[[ 0.6746, -0.5656, -0.2845,  ..., -0.7342,  0.9552,  1.2186],
          [-1.7622, -0.1387, -0.4715,  ..., -0.3910,  1.0678, -1.1081],
          [ 0.4437,  0.5277, -0.1378,  ..., -1.4772,  1.1539,  1.1894],
          ...,
          [ 0.1035,  0.2438, -1.0446,  ...,  1.4927,  0.0167, -0.2895],
          [ 0.2630, -0.0572, -0.1190,  ...,  0.5902,  0.3667, -0.2973],
          [-0.2895,  0.0656,  0.3984,  ...,  1.2987, -0.0803,  1.3355]],

         [[ 0.4910,  0.6885,  2.2282,  ...,  1.8876,  1.2659, -1.1326],
          [-2.4652,  1.8746, -0.1340,  ..., -0.1011,  0.1040, -1.2256],
          [ 1.0001, -1.4088, -1.2421,  ...,  1.0824, -0.8891,  0.1071],
          ...,
          [ 0.2784,  0.1146, -0.3480,  ..., -0.8340, -2.9496, -0.3927],
          [-1.4156, -0.4117,  0.7527,  ...,  0.4955, -0.0371, -0.3650],
          [-0.1038, -0.4548,  0.6555,  ..., -0.3951,  0.5098, -1.3594]],

         [[-2.4419, -0.1892, -1.3532,  ...,  2.1143,  0.6440, -0.7829],
          [-0.2887,  0.5106,  

In [10]:
torch.divide(1.0,x)

tensor([[[[   1.4824,   -1.7682,   -3.5144,  ...,   -1.3620,    1.0469,
              0.8206],
          [  -0.5675,   -7.2088,   -2.1210,  ...,   -2.5576,    0.9365,
             -0.9025],
          [   2.2538,    1.8950,   -7.2551,  ...,   -0.6770,    0.8666,
              0.8408],
          ...,
          [   9.6582,    4.1022,   -0.9573,  ...,    0.6699,   59.9630,
             -3.4537],
          [   3.8026,  -17.4942,   -8.4036,  ...,    1.6942,    2.7273,
             -3.3640],
          [  -3.4540,   15.2415,    2.5103,  ...,    0.7700,  -12.4558,
              0.7488]],

         [[   2.0367,    1.4525,    0.4488,  ...,    0.5298,    0.7899,
             -0.8829],
          [  -0.4056,    0.5334,   -7.4626,  ...,   -9.8906,    9.6175,
             -0.8159],
          [   0.9999,   -0.7098,   -0.8051,  ...,    0.9239,   -1.1248,
              9.3332],
          ...,
          [   3.5915,    8.7238,   -2.8734,  ...,   -1.1991,   -0.3390,
             -2.5462],
          [  -0.70

In [6]:
class FullDataset(Dataset):   
    def __getitem__(self, index):
        if index<256:
            return self.xgt[:,index,:,:], self.ymask[:,index,:,:]
        elif index<512:
            index = index-256
            return self.xgt[:,:,index,:], self.ymask[:,:,index,:]
        else:
            index = index-512
            return self.xgt[:,:,:,index], self.ymask[:,:,:,index]
    def __len__(self):
        return 768
    
index = 0
with open(f'/scratch/mrphys/pickled/fullDataset_{index}.pickle', 'rb') as f:
    dataset = pickle.load(f)

In [10]:
np.mean(dataset.xgt[0][100])

(-0.044717725-0.004932104j)

In [20]:
e = getComplexSlices(noisyPath[0])

In [17]:
np.mean(e[0][0,100])

(-0.03301484-0.0036413448j)

In [21]:
np.mean(e[0][0,100])

(-0.044717725-0.004932104j)